# Klasyfikacja tekstów

## Wprowadzenie
Ten przykład demonstruje jak można poradzić sobie z klasyfikacją tematyczną dokumentów stosując technikę "worek ze słowami". 

O tej technice więcej można przeczytać tu: [http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction]

W skrócie: 

* tekst jest zamieniany na  pojedyncze słowa. 
* ze słów otrzymanych dla dużej ilości tekstów tworzony jest słownik - wektor możliwych słów 
* dla konkretnego tekstu zliczana jest ilość wystąpień każdego ze słów
* te zliczenia mogą być normalizowane na różne sposoby

O metodzie tej mówi się "worek ze słowami", bo zaniedbujemy w niej kolejność słów w dokumencie i wszelkie korelacje między ich wzajemnymi pozycjami. 

Często stosowana wersja normalizacji to tzw. transformacja tf-idf (term frequency - inverse document frequency)[https://en.wikipedia.org/wiki/Tf–idf]. 

Transformacja ta zasadza się na iloczynie wagi i specyficzności danego słowa. W najprostszym przypadku:
* waga danego słowa jest proporcjonalna do częstości występowania słowa w dokumencie
* specyficzność słowa może być określona jako odwrotnie proporcjonalna do liczby dokumentów, w których występuje

Możliwe są różne wersje funkcji stosowanych w obu  składnikach.

## Dane
Dane pochodzą z 20 list dyskusyjnych.
Jako klasyfikator wykorzystamy naiwny klasyfikator Bayesa dla rozkładów wielorakich: <tt>MultinomialNB</tt>.

Przykład oparty na kodzie z: 
http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

* Authors: 
    Peter Prettenhofer <peter.prettenhofer@gmail.com>,
    Olivier Grisel <olivier.grisel@ensta.org>,
    Mathieu Blondel <mathieu@mblondel.org>,
    Lars Buitinck <L.J.Buitinck@uva.nl>,
* License: BSD 3 clause
* Adaptacja: Jarosław Żygierewicz

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import  MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

W całym zbiorze danych jest 20 list dyskusyjnych, tu wykorzystamy podzbiór.

Kategorie dla których zbudujemy klasyfikator:

In [ ]:
categories = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space'  ]

Ładujemy dane z newsgroups dataset dla wybranch kategorii. Od razu przygotowujemy dane testowe i treningowe.
Korzystamy z funkcji sklearn.datasets.fetch_20newsgroups http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

categories = data_train.target_names 

Zobaczmy jak wyglądają przykładowe dane o numerze 57

In [ ]:
id =57

Lista wiadomości

In [ ]:
print(data_train.data[id])        

Lista kodów tematycznych

In [ ]:
print(data_train.target[id])    

Nazwy kategorii odpowiadających kodom

In [ ]:
print(categories[data_train.target[id]] )

Upraszczamy nazewnictwo:

In [ ]:
y_train, y_test = data_train.target, data_test.target

Przekodowujemy wiadomości na wekotry cech.  Korzystamy z funkcji: [sklearn.feature_extraction.text.TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

In [ ]:
vectorizer = ....# stwórz instancje obiektu TfidfVectorizer
X_train = ....# naucz vctorizer słownika i przetransformuj dane uczące.  

Wypisz rozmiary danych treningowych

In [ ]:
print("Dane treningowe: n_samples: %d, n_features: %d" % X_train.shape)

Dane uczące są przechowywane w macierzy rzadkiej (sparse matrix). Proszę podejrzeć jak wyglądają tak przekodowane dane:

In [ ]:
print ...

 Wektoryzujemy też dane testowe:

In [ ]:
X_test = ...
print("Dane testowe: n_samples: %d, n_features: %d" % X_test.shape)

Odwrotne mapowanie z cech na słowa:

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names = np.asarray(feature_names)

Tworzymy instancję i uczymy klasyfikator MultinomialNB

In [ ]:
clf = ...
clf....

## Benchmark: tu będziemy korzystać z funkcji zaimplementowanych w 
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
pred = clf.... # obliczamy predykcję dla tekstów ze zbioru testowego
accur = ... # dokladność
print("dokładność:   %0.3f" % accur)
print("classification report:") # wypisz raport klasyfikacji 
print(...)

print("Macierz błędów") # wypisz macierz (confusion matrix)
print(...)

Wypiszemy teraz po 10 najbardziej znaczących słów w każdej klasie:

In [ ]:
print(u"top 10 haseł na klasę:")
for i, category in enumerate(categories):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print("%s: %s" % (category, " ".join(feature_names[top10])))